<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2022</h1>
<h1 style="font-size: 250%;">Problem Set #3 </h1>
<h1>Issued Thursday, 2/17/2022; Due Thursday, 3/8/2022</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will not be accepted. Feel free to get help from others, but **the work you submit in should be your own.**

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Jenna"
    last = "Levy"
    onyen = "jenlevy"
    pid = "730280089"

You will need a set of 13 base-pair <a href="http://csbio.unc.edu/mcmillan/Comp555S22/reads.fa" download="reads.fa">simulated reads</a> from a genome that will be used for assembly.
The cell below provides various functions and imports necessary for this problem set. Do not import any addtional packages. Also, make sure that you ***run the following cell***.

In [8]:
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into two lists. One of 
        headers and a second list of sequences.
        The ith index of each list correspond."""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

class Graph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}
        self.vertex = {i:v for i,v in enumerate(vlist)}
        self.edge = []
        self.edgelabel = []
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
    def hamiltonianPath(self):
        """ A Brute-force method for finding a Hamiltonian Path. 
        Basically, all possible N! paths are enumerated and checked
        for edges. Since edges can be reused there are no distictions
        made for *which* version of a repeated edge. """
        for path in itertools.permutations(sorted(self.index.values())):
            for i in range(len(path)-1):
                if ((path[i],path[i+1]) not in self.edge):
                    break
            else:
                return [self.vertex[i] for i in path]
        return []
    def SearchTree(self, path, verticesLeft):
        """ A recursive Branch-and-Bound Hamiltonian Path search. 
        Paths are extended one node at a time using only available
        edges from the graph. """
        if (len(verticesLeft) == 0):
            self.PathV2result = [self.vertex[i] for i in path]
            return True
        for v in verticesLeft:
            if (len(path) == 0) or ((path[-1],v) in self.edge):
                if self.SearchTree(path+[v], [r for r in verticesLeft if r != v]):
                    return True
        return False
    def hamiltonianPathV2(self):
        """ A wrapper function for invoking the Branch-and-Bound 
        Hamiltonian Path search. """
        self.PathV2result = []
        self.SearchTree([],sorted(self.index.values()))                
        return self.PathV2result
    def degrees(self):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in self.edge:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return inDegree, outDegree
    def verifyAndGetStart(self):
        inDegree, outDegree = self.degrees()
        start = 0
        end = 0
        for vert in self.vertex.keys():
            ins = inDegree.get(vert,0)
            outs = outDegree.get(vert,0)
            if (ins == outs):
                continue
            elif (ins - outs == 1):
                end = vert
            elif (outs - ins == 1):
                start = vert
            else:
                start, end = -1, -1
                break
        if (start >= 0) and (end >= 0):
            return start
        else:
            return -1
    def eulerianPath(self):
        graph = [(src,dst) for src,dst in self.edge]
        currentVertex = self.verifyAndGetStart()
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                for edge in graph:
                    try:
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print("There is no path!")
                    return False
        return path
    def eulerEdges(self, path):
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeList = []
        for i in range(len(path)-1):
            edgeList.append(self.edgelabel[edgeId[path[i],path[i+1]].pop()])            
        return edgeList
    def render(self, highlightPath=[]):
        """ Outputs a version of the graph that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeSet = set()
        for i in range(len(highlightPath)-1):
            src = self.index[highlightPath[i]]
            dst = self.index[highlightPath[i+1]]
            edgeSet.add(edgeId[src,dst].pop())
        result = ''
        result += 'digraph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for index, label in self.vertex.iteritems():
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (index, label)
        for i, e in enumerate(self.edge):
            src, dst = e
            result += '    N%d -> N%d' % (src, dst)
            label = self.edgelabel[i]
            if (len(label) > 0):
                if (i in edgeSet):
                    result += ' [label="%s", penwidth=3.0]' % (label)
                else:
                    result += ' [label="%s"]' % (label)
            elif (i in edgeSet):
                result += ' [penwidth=3.0]'                
            result += ';\n'                
        result += '    overlap=false;\n'
        result += '}\n'
        return result

**Problem #1:** Construct a minimal string of DNA nucleotides includes every 4-base k-mer exactly once *except* "ACGT", which appears twice. How long is your string? How long is the minimal string of DNA nucleotides that includes all 4-mers except "ACGT"?

In [9]:
# This cell is provided for work on problem 1. It will not be graded.

from itertools import permutations 
from itertools import product

all_kmers = ["".join(tup) for tup in itertools.product("ACGT", repeat = 4)] + ["ACGT"]
#all_kmers = ["".join(tup) for tup in itertools.product("ACGT", repeat = 4) if ''.join(tup) != "ACGT"] 
print(len(all_kmers))

nodes = set([k[:-1] for k in all_kmers] + [k[1:] for k in all_kmers])
print(len(nodes))

G = Graph(nodes)
for k in all_kmers:
    G.addEdge(k[:-1], k[1:], label = k)
    
path = G.eulerianPath()
print(path)
print(len(path))

edgeList = G.eulerEdges(path)
print(edgeList)
print(len(edgeList))

twoACGT = edgeList[0] + ''.join([e[-1] for e in edgeList[1:]])
print(len(twoACGT))
print(twoACGT)




257
64
[10, 20, 8, 22, 22, 58, 55, 16, 22, 7, 14, 8, 58, 2, 60, 16, 58, 10, 34, 25, 16, 7, 43, 25, 3, 55, 3, 2, 30, 60, 3, 10, 48, 29, 8, 7, 62, 29, 40, 55, 18, 14, 40, 2, 11, 20, 40, 10, 39, 36, 38, 22, 4, 61, 28, 61, 38, 58, 52, 63, 38, 7, 33, 36, 57, 55, 19, 61, 57, 2, 45, 63, 57, 10, 39, 31, 1, 16, 4, 26, 50, 63, 28, 26, 1, 3, 52, 5, 50, 5, 1, 18, 43, 12, 60, 18, 62, 37, 25, 18, 33, 31, 51, 60, 19, 26, 51, 30, 30, 11, 34, 12, 30, 45, 5, 51, 11, 48, 37, 12, 11, 39, 47, 24, 47, 6, 8, 4, 13, 32, 29, 59, 13, 24, 36, 28, 13, 6, 40, 52, 21, 32, 37, 27, 21, 24, 31, 50, 21, 6, 35, 14, 35, 43, 0, 20, 35, 62, 41, 41, 15, 49, 46, 46, 17, 24, 49, 17, 32, 15, 47, 32, 41, 29, 35, 33, 47, 42, 25, 19, 13, 42, 12, 45, 21, 42, 0, 34, 0, 48, 41, 37, 0, 39, 49, 23, 38, 4, 53, 17, 6, 59, 53, 46, 23, 28, 53, 23, 57, 52, 44, 17, 42, 27, 44, 46, 54, 50, 44, 23, 9, 14, 59, 61, 9, 43, 27, 63, 9, 62, 15, 36, 9, 33, 49, 54, 1, 19, 53, 54, 51, 45, 44, 54, 56, 20, 59, 26, 56, 34, 27, 5, 56, 48, 15, 31, 56, 39]


**Problem #2:** A file of 13-mers simulating short reads from a genome can be <a href="http://csbio.unc.edu/mcmillan/Comp555S22/reads.fa" download="reads.fa">downloaded here</a>. How many distinct nodes appear in the De Bruijn graph that represents these 13-mers as edges? How many nodes are semi-balenced? How many nodes are balanced? How many are balanced with both in-degrees and out-degrees equal to 1?

In [4]:
# This cell is provided for your own use in answering problem 4. It will not be graded.
header, seq = loadFasta("reads (1).fa")

#print(len(seq))
#print(seq[0])

nodes = set([k[1:-1] for k in seq] + [k[2:] for k in seq])

print(len(nodes))

G = Graph(nodes)

for k in seq:
    G.addEdge(k[1:-1], k[2:], label=k[1:])
    
indegree, outdegree = G.degrees()

#print(len(indegree))
#print(len(outdegree))


semi = 0
balanced = 0
oneOne = 0

for index in G.vertex: 
    ins = indegree.get(index,0)
    outs = outdegree.get(index,0)
    if abs(ins-outs) == 1:
        semi+=1
    elif ins==outs:
        balanced += 1
        if ins == 1:
            oneOne += 1
        ##else:
            ##print(index, ins, outs)
            
print(semi)
print(balanced)
print(oneOne)


29777
2
29775
29682


**Problem #3:** What is the length of the Eulerian path that can be constructed in the De Bruijn graph described in Problem #2?. How long did it take to assemble it using Jupyter %time? Find the starting position of this assembled sequence in that genome.

In [53]:
# This cell is provided for your own use in answering problem 3. It will not be graded.

header, seq = loadFasta("reads (1).fa")

print(len(seq))
print(seq[0])

nodes = set([k[1:-1] for k in seq] + [k[2:] for k in seq])

print(len(nodes))

G = Graph(nodes)

for k in seq:
    G.addEdge(k[1:-1], k[2:], label=k[1:])
    
Eul_G = G.eulerianPath()
%timeit G.eulerianPath()
print(len(Eul_G))

edgeList = G.eulerEdges(Eul_G)
#print(edgeList)
print(len(edgeList))



29891
+AAAAAAAAAAAAA
29777
55.3 s ± 657 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
29892
29891


**Problem #4:** Construct a suffix array of the <a href="http://csbio.unc.edu/mcmillan/Comp555S22/SARS-COV2Wuhan.fasta" download="SARS-COV2Wuhan.fasta">SARS-COV2Wuhan.fasta</a> with the initial '+' removed and a '$' appended. Use it to find all the positions of the substring "ATGCAT" within this substring, and their interval within the suffix array. How many times does "CAT" appear in the specified substring?

In [11]:
# This cell is provided for your own use in answering problem 4. It will not be graded.

header, seq = loadFasta("SARS-COV-2Wuhan.fasta")

def suffixArray(string):
    return sorted(range(len(string)), key=lambda x: string[x:])

def findLast(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:suffixarray[middle]+len(pattern)] <= pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo


def findFirst(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:] < pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo


dna2 = ''.join(seq)
dna2 = dna2.replace('+','')
dna2 = dna2 + '$'

sa= suffixArray(dna2)


first = findFirst("ATGCAT", dna2, sa)

##print(first, sa[first], dna2[sa[first]:])

last = findLast("ATGCAT", dna2, sa)
print(first, last)
for suffix in sa[first:last]:     # recall "first" was found on the previous slide
    print("%3d: %s" % (suffix, dna2[suffix:]))
print(last - first, "times")


7684 7693
12536: ATGCATCAGCATTGTGGGAAATCCAACAGGTTGTAGATGCAGATAGTAAAATTGTTCAACTTAGTGAAATTAGTATGGACAATTCACCTAATTTAGCATGGCCTCTTATTGTAACAGCTTTAAGGGCCAATTCTGCTGTCAAATTACAGAATAATGAGCTTAGTCCTGTTGCACTACGACAGATGTCTTGTGCTGCCGGTACTACACAAACTGCTTGCACTGATGACAATGCGTTAGCTTACTACAACACAACAAAGGGAGGTAGGTTTGTACTTGCACTGTTATCCGATTTACAGGATTTGAAATGGGCTAGATTCCCTAAGAGTGATGGAACTGGTACTATCTATACAGAACTGGAACCACCTTGTAGGTTTGTTACAGACACACCTAAAGGTCCTAAAGTGAAGTATTTATACTTTATTAAAGGATTAAACAACCTAAATAGAGGTATGGTACTTGGTAGTTTAGCTGCCACAGTACGTCTACAAGCTGGTAATGCAACAGAAGTGCCTGCCAATTCAACTGTATTATCTTTCTGTGCTTTTGCTGTAGATGCTGCTAAAGCTTACAAAGATTATCTAGCTAGTGGGGGACAACCAATCACTAATTGTGTTAAGATGTTGTGTACACACACTGGTACTGGTCAGGCAATAACAGTTACACCGGAAGCCAATATGGATCAAGAATCCTTTGGTGGTGCATCGTGTTGTCTGTACTGCCGTTGCCACATAGATCATCCAAATCCTAAAGGATTTTGTGACTTAAAAGGTAAGTATGTACAAATACCTACAACTTGTGCTAATGACCCTGTGGGTTTTACACTTAAAAACACAGTCTGTACCGTCTGCGGTATGTGGAAAGGTTATGGCTGTAGTTGTGATCAACTCCGCGAACCCATGCTTCAGTCAGCTGATGCACAATCGTTTTTAAACGGGTTTGCGGTGTAAGTGCAGCCCGTCTTACACCGTGCGGCACAGGCACTA

**Problem #5:** Use your suffix array from Problem #4 to construct a BWT of SARS-COV2Wuhan. Once the BWT is found encode any run of more than 1 base as an integer followed by the base. What is the total length of this compressed BWT? What is the average run-length of 'A', 'C', 'G' and 'T' nucleotides?  

In [52]:
# This cell is provided for your own use in answering problem 5. It will not be graded.
import collections
from collections import Counter

header, seq = loadFasta("SARS-COV-2Wuhan.fasta")

def suffixArray(string):
    return sorted(range(len(string)), key=lambda x: string[x:])

def BWTfromSuffixArray(text, suffixarray):
    return ''.join(text[i-1] for i in suffixarray)

dna2 = ''.join(seq)
dna2 = dna2.replace('+','')
dna2 = dna2 + '$'

sa= suffixArray(dna2)
bwt_w = BWTfromSuffixArray(dna2, sa)
compress_length = len(bwt_w)
compressedBWT = ""

counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0, '$': 0}
runs = {'A': 0, 'C': 0, 'G': 0, 'T': 0, '$': 0}

i = 0

while i < compress_length:
    counter = 1
    while i<(compress_length-1) and bwt_w[i] == bwt_w[i+1]:
        counter += 1
        i += 1
    if counter == 1:
        compressedBWT += str(bwt_w[i])
        counts[bwt_w[i]] += 1
    else:
        compressedBWT += str(counter) + str(bwt_w[i])
        counts[bwt_w[i]] += counter
    runs[bwt_w[i]] += 1

    i+= 1
    
print(len(compressedBWT))

for key in runs:
    print(key, counts[key]/runs[key])

27321
A 1.4533354974841746
C 1.263109475620975
G 1.2605891206192217
T 1.50920245398773
$ 1.0


---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 